# 의사결정나무 만들기

## Data Load

In [69]:
import pandas as pd 
import numpy as np

In [70]:
pd_data = pd.read_csv('https://raw.githubusercontent.com/AugustLONG/ML01/master/01decisiontree/AllElectronics.csv')
pd_data.drop("RID",axis=1, inplace = True) #RID는 그냥 순서라서 삭제
pd_data

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
6,middle_aged,low,yes,excellent,yes
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes


# 1. Gini 계수 구하기

Input: df(데이터), label(타겟변수명)

In [100]:
def get_gini(df, label): 
    counts = list(df.groupby(label).count().iloc[:,0]) # 나온 횟수만 추출
    total = len(df) # 데이터프레임 길이 = 전체 데이터 개수
    
    gini = 1
    # 지니계수 계산하기
    for p in counts:
        gini -= (p/total)**2
    return gini   

In [101]:
get_gini(pd_data,'class_buys_computer')

0.4591836734693877

# 2. Feature의 Class를 이진 분류로 만들기
 ## ex) {A,B,C} -> ({A}, {B,C}), ({B}, {A,C}), ({C}, {A,B})

Input: df(데이터), attribute(Gini index를 구하고자 하는 변수명)

In [58]:
def get_binary_split(df, attribute):
    # 만들 수 있는 피처 조합 생성
    features = [[]]
    feature = df[attribute].unique()
    for f in feature:
        for fs in features:
            features = features + [list(fs) + [f]]
    
    # 아무 피처도 포함하지 않는 경우와 모든 피처를 포함하는 경우를 제외
    return features[1:len(features)-1]

In [59]:
get_binary_split(pd_data,"age")

[['youth'],
 ['middle_aged'],
 ['youth', 'middle_aged'],
 ['senior'],
 ['youth', 'senior'],
 ['middle_aged', 'senior']]

# 3. 모든 이진분류의 경우의 Gini index를 구하는 함수 만들기

In [113]:
def get_attribute_gini_index(df, attribute, label):
    result = {}
    binarysplit = get_binary_split(df, attribute) #이진분류
    gini_1 = gini_2 = None
    
    for i in range(len(binarysplit)):
        # 분류 결과의 집합에 원소가 하나인 경우
        if len(binarysplit[i]) == 1 :  
            d1 = df[df[attribute] == str(binarysplit[i][0])]
            d2 = df[df[attribute] != str(binarysplit[i][0])]
            
        # 분류 결과의 집합에 원소가 하나 이상일 경우
        elif len(binarysplit[i]) > 1:  
            splited = binarysplit[i]
            d1 = pd.DataFrame()
            
            #집합 원소들을 클래스로 가지는 데이터 분류
            for s in splited:
                d = df[df[attribute] == s]
                d1 = pd.concat([d1, d])
                
            d2 = pd_data.drop(d1.index) #나머지 데이터
            
            #원소가 2개 이상('youth','senior')일 때 "youth_senior"처럼 한 단어로 이름 변경
            binarysplit[i] = (binarysplit[i][0] + "_" + binarysplit[i][1]).split()  
        
        # 값 구하기
        gini_1 = get_gini(d1, label) #해당데이터 지니계수
        gini_2 = get_gini(d2, label) #나머지 데이터 지니계수
        nrow = d1.shape[0] # split 후 개수 저장 (result에 사용하기 위함)   
        
        #최종 gini계수 dictionary형태로 저장
        result[binarysplit[i][0]] = (nrow/df.shape[0])*gini_1 + (1-(nrow/df.shape[0]))*gini_2   
        
    return result

In [108]:
get_attribute_gini_index(pd_data, "age", "class_buys_computer")

{'youth': 0.3936507936507936,
 'middle_aged': 0.35714285714285715,
 'youth_middle_aged': 0.4571428571428571,
 'senior': 0.4571428571428571,
 'youth_senior': 0.35714285714285715,
 'middle_aged_senior': 0.39365079365079364}

여기서 가장 작은 Gini index값을 가지는 class를 기준으로 split해야 한다.

In [62]:
min(get_attribute_gini_index(pd_data, "age", "class_buys_computer").items())

('middle_aged', 0.35714285714285715)

In [63]:
## 1. 변수 ‘income’의 이진분류 결과
get_binary_split(pd_data, "income")

[['high'],
 ['medium'],
 ['high', 'medium'],
 ['low'],
 ['high', 'low'],
 ['medium', 'low']]

In [64]:
## 2. 분류를 하는 데 가장 중요한 변수를 선정하고, 해당 변수의 Gini index를 제시
ginis = {'age':min(get_attribute_gini_index(pd_data, "age", "class_buys_computer").values()),
         "income":min(get_attribute_gini_index(pd_data, "income", "class_buys_computer").values()),
         'student': min(get_attribute_gini_index(pd_data, "student", "class_buys_computer").values()),
        "crdit_rating": min(get_attribute_gini_index(pd_data, "credit_rating", "class_buys_computer").values())}
print(min(ginis),ginis[min(ginis)])

age 0.35714285714285715


In [68]:
## 3. 문제 2에서 제시한 feature로 DataFrame을 split한 후 
##    나눠진 2개의 DataFrame에서 각각 다음으로 중요한 변수를 선정하고 해당 변수의 Gini index를 제시

data1 = pd_data[pd_data["age"]=="middle_aged"] 
data2 = pd_data[pd_data["age"]!="middle_aged"]
 
ginis1 = {"income":min(get_attribute_gini_index(data1, "income", "class_buys_computer").values()), 
         'student': min(get_attribute_gini_index(data1, "student", "class_buys_computer").values()),
        "credit_rating": min(get_attribute_gini_index(data1, "credit_rating", "class_buys_computer").values())}

ginis2 = {'age':min(get_attribute_gini_index(data2, "age", "class_buys_computer").values()),
         "income":min(get_attribute_gini_index(data2, "income", "class_buys_computer").values()),
         'student': min(get_attribute_gini_index(data2, "student", "class_buys_computer").values()),
        "credit_rating": min(get_attribute_gini_index(data2, "credit_rating", "class_buys_computer").values())}
 
print(min(ginis1.items(), key=lambda x: x[1]))
print(min(ginis2.items(), key=lambda x: x[1]))

('income', 0.0)
('student', 0.31999999999999984)


참고: http://www.datamarket.kr/xe/board_BoGi29/52721